# Project Report II: Data Cleaning and Exploratory Data Analysis
___
**Brief:**<br>
In this report, I'm going to finish the data cleaning and organization needed for my project, and then move on to exploratory data analysis. Lets pick up where we left off last time.

**Sections:**<br>
1. [Reading In An Email](#reading_email)
    - [Imports, Choosing A File, Reading In](#re_1)
    - [Appending Blanks Together](#re_2)
___

## Reading In An Email
In this section, we'll be picking up where we left off in the last report.

<a id='re_1'>

*Imports, Choosing A File, Reading In*

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import nltk
from nltk.corpus import PlaintextCorpusReader as cr
import pandas as pd
import numpy as np

In [2]:
corpus_root = '../../../../../Enron_Emails/maildir/'
all_allen_p = 'allen-p/all_documents'
all_files = cr(corpus_root + all_allen_p, ".*")
print(len(all_files.fileids()))

628


In [3]:
text = ""
file = open(corpus_root + all_allen_p + "/1")
for line in file:
    text += line
file.close()
email = text.split('\n')

In [4]:
email

['Message-ID: <29790972.1075855665306.JavaMail.evans@thyme>',
 'Date: Wed, 13 Dec 2000 18:41:00 -0800 (PST)',
 'From: 1.11913372.-2@multexinvestornetwork.com',
 'To: pallen@enron.com',
 "Subject: December 14, 2000 - Bear Stearns' predictions for telecom in Latin",
 ' America',
 'Mime-Version: 1.0',
 'Content-Type: text/plain; charset=us-ascii',
 'Content-Transfer-Encoding: 7bit',
 'X-From: Multex Investor <1.11913372.-2@multexinvestornetwork.com>',
 'X-To: <pallen@enron.com>',
 'X-cc: ',
 'X-bcc: ',
 'X-Folder: \\Phillip_Allen_Dec2000\\Notes Folders\\All documents',
 'X-Origin: Allen-P',
 'X-FileName: pallen.nsf',
 '',
 "In today's Daily Update you'll find free reports on",
 'America Online (AOL), Divine Interventures (DVIN),',
 'and 3M (MMM); reports on the broadband space, Latin',
 'American telecom, and more.',
 '',
 "For free research, editor's picks, and more come to the Daily Investor:",
 'http://www.multexinvestor.com/AF004627/magazinecover.asp?promo=unl&d=20001214#',
 'investor

<a id='re_2'>

***Appending Blanks Together***<br>
So, the problem we ran into last time. Lets see if a simple "add and delete" works.